In [0]:

# Minimal fix: Use Python to read JSON from Unity Catalog volume and save as table
json_path = "/Volumes/dev/club_db/data/json/simple_json.json"
df = spark.read.json(json_path)
df.write.saveAsTable("dev.club_db.bronze_simple_json")

In [0]:
%sql
select * from dev.club_db.bronze_simple_json


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW raw_employees AS
SELECT *
FROM read_files(
  "/Volumes/dev/club_db/data/json/nested_json.json",
  format => "json",
  multiline => true
);

In [0]:
%sql
select * from raw_employees

In [0]:
from pyspark.sql.types import StructType, StructField, LongType, StringType

json_path1 = "/Volumes/dev/club_db/data/json/nested_json.json"
schema = StructType([
    StructField('age', LongType(), True),
    StructField('id', LongType(), True),
    StructField('name', StringType(), True)
])
df1 = spark.read.schema(schema).option('mode', 'PERMISSIVE').json(json_path1)
df1.write.saveAsTable("dev.club_db.bronze_nested_json")

In [0]:
%sql
select *  from dev.club_db.bronze_nested_json